画像・文書ファイル（学習用）　→　28×28のデータに変換

In [ ]:


from pathlib import Path

# 対象を切り替える： "train_test" または "verify"
TARGET_SPLIT = "train_test"   # ← 必要に応じて "verify" に変更

IN_DIR  = Path("uploads") / TARGET_SPLIT
OUT_DIR = Path("data") / TARGET_SPLIT
OUT_DIR.mkdir(parents=True, exist_ok=True)

print("対象フォルダ:", IN_DIR)
print("保存先:", OUT_DIR)
import re
import numpy as np
from PIL import Image, ImageOps
import fitz  # PyMuPDF

# ======= 20枠(4×5) 固定 =======
ROWS, COLS = 4, 5
LABELS_20 = [0,1,2,3,4,
             5,6,7,8,9,
             0,1,2,3,4,
             5,6,7,8,9]

# ======= パラメータ（授業で安定しやすい初期値） =======
ZOOM_PDF = 3.0
INNER_MARGIN_PX = 10      # 枠線が残るなら 12-16 に上げる
LINE_DARK_THRESH = 200    # 枠線が薄いなら 210-230 に上げる
PEAK_REL = 0.55           # 線検出の厳しさ（0.45-0.70で調整）
MIN_BAND = 3              # 線として認める連続px幅
BW_THR = 160              # 薄い字なら 130-150 に下げる
INVERT = True             # 白地に黒字想定。逆なら False

ALLOWED_EXT = {".jpg", ".jpeg", ".png", ".pdf"}

def _safe_stem(p: Path) -> str:
    s = p.stem
    s = re.sub(r"[^0-9A-Za-z_\-]+", "_", s)  # 変な文字を _
    s = s.strip("_")
    return s or "file"

def load_first_page_as_image(path: Path) -> Image.Image:
    suf = path.suffix.lower()
    if suf == ".pdf":
        doc = fitz.open(str(path))
        if len(doc) < 1:
            raise RuntimeError("PDFにページがありません。")
        mat = fitz.Matrix(ZOOM_PDF, ZOOM_PDF)
        pix = doc[0].get_pixmap(matrix=mat, alpha=False)
        return Image.frombytes("RGB", [pix.width, pix.height], pix.samples)
    else:
        return Image.open(path).convert("RGB")

def _bands_from_projection(proj: np.ndarray, thresh: float, min_band: int):
    above = proj >= thresh
    bands = []
    i, n = 0, len(proj)
    while i < n:
        if not above[i]:
            i += 1
            continue
        j = i + 1
        while j < n and above[j]:
            j += 1
        if (j - i) >= min_band:
            bands.append((i, j, float(proj[i:j].mean())))
        i = j
    return bands

def _pick_n_best_centers(bands, n):
    if len(bands) < n:
        return []
    bands = sorted(bands, key=lambda x: x[2], reverse=True)[:n]
    centers = [int((a + b) // 2) for a, b, _ in bands]
    centers.sort()
    return centers

def detect_grid_lines(page_img: Image.Image):
    gray = page_img.convert("L")
    arr = np.array(gray)

    # 黒っぽい画素を線として数える（枠線が一番効く）
    mask = arr < LINE_DARK_THRESH
    col_sum = mask.sum(axis=0).astype(np.float32)
    row_sum = mask.sum(axis=1).astype(np.float32)

    col_thresh = PEAK_REL * float(col_sum.max())
    row_thresh = PEAK_REL * float(row_sum.max())

    x_bands = _bands_from_projection(col_sum, col_thresh, MIN_BAND)
    y_bands = _bands_from_projection(row_sum, row_thresh, MIN_BAND)

    x_lines = _pick_n_best_centers(x_bands, COLS + 1)  # 縦線6本
    y_lines = _pick_n_best_centers(y_bands, ROWS + 1)  # 横線5本

    if len(x_lines) != COLS + 1 or len(y_lines) != ROWS + 1:
        raise RuntimeError(f"グリッド線検出失敗（必要: x=6,y=5 / 実際: x={len(x_lines)}, y={len(y_lines)}）")

    return x_lines, y_lines

def split_by_lines(page_img: Image.Image, x_lines, y_lines):
    crops = []
    for r in range(ROWS):
        y0 = y_lines[r] + INNER_MARGIN_PX
        y1 = y_lines[r + 1] - INNER_MARGIN_PX
        for c in range(COLS):
            x0 = x_lines[c] + INNER_MARGIN_PX
            x1 = x_lines[c + 1] - INNER_MARGIN_PX
            crops.append(page_img.crop((x0, y0, x1, y1)))
    return crops

def split_equal_grid(page_img: Image.Image):
    """フォールバック：等間隔分割（授業が止まらない保険）"""
    w, h = page_img.size
    cell_w = w / COLS
    cell_h = h / ROWS
    crops = []
    for r in range(ROWS):
        for c in range(COLS):
            x0 = int(c * cell_w) + INNER_MARGIN_PX
            y0 = int(r * cell_h) + INNER_MARGIN_PX
            x1 = int((c + 1) * cell_w) - INNER_MARGIN_PX
            y1 = int((r + 1) * cell_h) - INNER_MARGIN_PX
            crops.append(page_img.crop((x0, y0, x1, y1)))
    return crops

def to_28x28(img: Image.Image) -> Image.Image:
    g = img.convert("L")
    if INVERT:
        g = ImageOps.invert(g)

    arr = np.array(g)
    bw = (arr > BW_THR).astype(np.uint8) * 255
    bw_img = Image.fromarray(bw, mode="L")

    # 余白除去（白以外があればそこに詰める）
    arr2 = np.array(bw_img)
    ys, xs = np.where(arr2 < 255)
    if len(xs) > 0 and len(ys) > 0:
        x0, x1 = xs.min(), xs.max()
        y0, y1 = ys.min(), ys.max()
        bw_img = bw_img.crop((x0, y0, x1 + 1, y1 + 1))

    bw_img = ImageOps.pad(
        bw_img, (28, 28),
        method=Image.Resampling.NEAREST,
        color=255, centering=(0.5, 0.5)
    )
    return bw_img

def convert_one_file(in_path: Path):
    """
    1ファイルを変換して返す
    戻り:
      info: dict（成功/失敗/注意）
      X: (20,784) float32
      y: (20,) int64
    """
    page_img = load_first_page_as_image(in_path)

    info = {
        "input": str(in_path),
        "mode": None,                 # "line-detect" or "fallback"
        "success": False,
        "message": "",
        "warning": None,
        "error": None,
    }

    try:
        x_lines, y_lines = detect_grid_lines(page_img)
        crops = split_by_lines(page_img, x_lines, y_lines)
        info["mode"] = "line-detect"
        info["success"] = True
        info["message"] = "成功（線検出で分割）"
    except Exception as e:
        # 線検出失敗 → フォールバック
        crops = split_equal_grid(page_img)
        info["mode"] = "fallback"
        info["success"] = True  # 処理自体は継続できる
        info["warning"] = f"{type(e).__name__}: {e}"
        info["message"] = "注意（線検出に失敗 → 等間隔分割で代替）"

    if len(crops) != ROWS * COLS:
        info["success"] = False
        info["error"] = f"分割数が不正です: {len(crops)}（期待: {ROWS*COLS}）"
        info["message"] = "失敗（分割に失敗）"
        return info, None, None

    imgs28 = [to_28x28(c) for c in crops]
    X = np.stack([np.array(im, dtype=np.float32).reshape(-1) / 255.0 for im in imgs28], axis=0)
    y = np.array(LABELS_20, dtype=np.int64)
    return info, X, y

def batch_convert(in_dir: Path, out_dir: Path):
    in_dir = Path(in_dir)
    out_dir = Path(out_dir)
    out_dir.mkdir(parents=True, exist_ok=True)

    targets = sorted([p for p in in_dir.glob("*") if p.suffix.lower() in ALLOWED_EXT])
    if not targets:
        print("対象ファイルがありません:", in_dir)
        return

    ok_count = 0
    warn_count = 0
    fail_count = 0

    for i, p in enumerate(targets, 1):
        info, X, y = convert_one_file(p)

        # 日本語の成功/失敗表示
        head = f"[{i:03d}/{len(targets):03d}] {p.name}"
        if info["success"]:
            ok_count += 1
            if info["mode"] == "fallback":
                warn_count += 1
                print(head, ":", info["message"])
                print("  → 撮り直してください（紙全体・真上・影なし・ピンボケなし）")
                print("  補足:", info["warning"])
            else:
                print(head, ":", info["message"])
        else:
            fail_count += 1
            print(head, ":", info["message"])
            print("  → 撮り直してください（紙全体が写っていない/傾きが大きい可能性）")
            print("  エラー:", info["error"])
            continue

        # 保存（入力ファイルごとに1つのNPZ）
        # ファイル名問題を避ける：連番+安全なstem
        safe = _safe_stem(p)
        out_path = out_dir / f"{i:03d}_{safe}.npz"
        np.savez_compressed(
            out_path,
            X=X.astype(np.float32),
            y=y.astype(np.int64),
            input_name=p.name,
            mode=info["mode"],
            warning="" if info["warning"] is None else info["warning"],
        )

    print("\n===== 変換まとめ =====")
    print("対象:", len(targets))
    print("保存成功:", ok_count)
    print("うち注意（撮り直し推奨/fallback）:", warn_count)
    print("失敗:", fail_count)
    print("保存先:", out_dir)
batch_convert(IN_DIR, OUT_DIR)


手書き数字認識モデル作成

In [ ]:
from pathlib import Path
import numpy as np

# ==== 入力データ（変換後）====
TRAIN_SPLIT_DIR = Path("data/train_test")   # 学習・評価用（npzが入っている）
MODEL_DIR = Path("data/models")
MODEL_DIR.mkdir(parents=True, exist_ok=True)

# ==== train:test = 3:1 ====
TEST_RATIO = 0.25  # 1/4 = 25%

# ==== LogisticRegression パラメータ（いじりたいところ）====
LOGREG_PARAMS = {
    "C": 2.0,                 # 正則化の強さ（大きいほど弱い正則化）
    "penalty": "l2",          # 基本は l2
    "solver": "lbfgs",        # 安定: lbfgs（l2向け）
    "max_iter": 2000,         # 収束しない場合は増やす
    "class_weight": None,     # 不均衡が気になるなら "balanced"
    "random_state": 42,       # 再現性
}

# ==== 保存先（後段の推論コードで使う）====
MODEL_PATH = MODEL_DIR / "digits_logreg.joblib"

# ==== 日本語フォント（リポジトリ同梱・強制設定）====
FONT_PATH = Path("fonts/NotoSansCJKjp-Regular.otf")  # ← あなたが配置したパス


def load_npz_dataset(npz_dir: Path):
    npz_files = sorted(npz_dir.glob("*.npz"))
    if not npz_files:
        raise FileNotFoundError(f"npzが見つかりません: {npz_dir}")

    X_list, y_list = [], []
    meta = []

    for p in npz_files:
        z = np.load(p, allow_pickle=True)
        X = z["X"].astype(np.float32)
        y = z["y"].astype(np.int64)

        if X.ndim != 2 or X.shape[1] != 784:
            raise ValueError(f"Xの形が想定外: {p} -> {X.shape}")
        if y.ndim != 1 or len(y) != len(X):
            raise ValueError(f"yの形が想定外: {p} -> y={y.shape}, X={X.shape}")

        X_list.append(X)
        y_list.append(y)

        input_name = str(z["input_name"]) if "input_name" in z else p.name
        mode = str(z["mode"]) if "mode" in z else "unknown"
        meta.append((p.name, input_name, mode))

    X_all = np.concatenate(X_list, axis=0)
    y_all = np.concatenate(y_list, axis=0)

    return X_all, y_all, npz_files, meta


# ==========================
# Load dataset
# ==========================
X, y, npz_files, meta = load_npz_dataset(TRAIN_SPLIT_DIR)

print("=== 読み込み完了 ===")
print("npz数:", len(npz_files))
print("X:", X.shape, "y:", y.shape)
print("ラベル分布:", {i: int((y == i).sum()) for i in range(10)})


# ==========================
# Train / Test split + Train
# ==========================
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import joblib

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=TEST_RATIO,
    random_state=LOGREG_PARAMS.get("random_state", 42),
    stratify=y
)

print("=== 分割 ===")
print("train:", X_train.shape, "test:", X_test.shape)

model = LogisticRegression(
    C=LOGREG_PARAMS["C"],
    penalty=LOGREG_PARAMS["penalty"],
    solver=LOGREG_PARAMS["solver"],
    max_iter=LOGREG_PARAMS["max_iter"],
    class_weight=LOGREG_PARAMS["class_weight"],
    random_state=LOGREG_PARAMS["random_state"],
    multi_class="auto",
)

model.fit(X_train, y_train)

joblib.dump(model, MODEL_PATH)
print("=== 学習完了 ===")
print("保存先:", MODEL_PATH)


# ==========================
# Force Japanese font for matplotlib
# ==========================
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

if not FONT_PATH.exists():
    raise FileNotFoundError(
        f"日本語フォントが見つかりません: {FONT_PATH}\n"
        "GitHubリポジトリの fonts/ に NotoSansCJKjp-Regular.otf を置いてください。"
    )

# フォント登録 → 強制適用
fm.fontManager.addfont(str(FONT_PATH))
font_prop = fm.FontProperties(fname=str(FONT_PATH))

plt.rcParams["font.family"] = font_prop.get_name()
plt.rcParams["axes.unicode_minus"] = False  # マイナスの文字化け対策

print("=== 日本語フォント設定完了 ===")
print("FONT_PATH:", FONT_PATH)
print("font name:", font_prop.get_name())


# ==========================
# Evaluation (overall + per digit) + plots
# ==========================
from sklearn.metrics import accuracy_score, confusion_matrix

y_pred = model.predict(X_test)

acc = accuracy_score(y_test, y_pred)
print("\n=== テスト結果（全体）===")
print(f"全体正答率: {acc:.4f}  (test n={len(y_test)})")

print("\n=== テスト結果（数字ごと）===")
per_digit_acc = {}
per_digit_n = {}
for d in range(10):
    mask = (y_test == d)
    n = int(mask.sum())
    per_digit_n[d] = n
    if n == 0:
        per_digit_acc[d] = np.nan
        print(f"{d}: test数=0")
    else:
        a = float((y_pred[mask] == y_test[mask]).mean())
        per_digit_acc[d] = a
        print(f"{d}: 正答率={a:.4f}  (test数={n})")

digits = list(range(10))
vals = [per_digit_acc[d] for d in digits]

# ---- 棒グラフ：数字ごとの正答率 ----
plt.figure(figsize=(10, 4))
plt.bar(digits, [0 if np.isnan(v) else v for v in vals])
plt.ylim(0, 1.0)
plt.xticks(digits, [str(d) for d in digits])
plt.xlabel("数字")
plt.ylabel("正答率")
plt.title("数字ごとの正答率（ロジスティック回帰）")
plt.grid(axis="y", linestyle="--", alpha=0.4)

# test数も注記（小さく表示）
for i, d in enumerate(digits):
    n = per_digit_n[d]
    v = vals[i]
    if not np.isnan(v):
        plt.text(d, v + 0.02, f"n={n}", ha="center", va="bottom", fontsize=9)

plt.tight_layout()
plt.show()

# ---- 混同行列 ----
cm = confusion_matrix(y_test, y_pred, labels=digits)

plt.figure(figsize=(6, 5))
plt.imshow(cm, interpolation="nearest")
plt.title("混同行列（テスト）")
plt.xlabel("予測")
plt.ylabel("正解")
plt.xticks(digits, digits)
plt.yticks(digits, digits)

for i in range(10):
    for j in range(10):
        v = cm[i, j]
        if v != 0:
            plt.text(j, i, str(v), ha="center", va="center", fontsize=9)

plt.colorbar()
plt.tight_layout()
plt.show()


In [ ]:
画像・文書ファイル（検証用）　→　28×28のデータに変換

In [ ]:


from pathlib import Path

# 対象を切り替える： "train_test" または "verify"
TARGET_SPLIT = "train_test"   # ← 必要に応じて "verify" に変更

IN_DIR  = Path("uploads") / TARGET_SPLIT
OUT_DIR = Path("data") / TARGET_SPLIT
OUT_DIR.mkdir(parents=True, exist_ok=True)

print("対象フォルダ:", IN_DIR)
print("保存先:", OUT_DIR)
import re
import numpy as np
from PIL import Image, ImageOps
import fitz  # PyMuPDF

# ======= 20枠(4×5) 固定 =======
ROWS, COLS = 4, 5
LABELS_20 = [0,1,2,3,4,
             5,6,7,8,9,
             0,1,2,3,4,
             5,6,7,8,9]

# ======= パラメータ（授業で安定しやすい初期値） =======
ZOOM_PDF = 3.0
INNER_MARGIN_PX = 10      # 枠線が残るなら 12-16 に上げる
LINE_DARK_THRESH = 200    # 枠線が薄いなら 210-230 に上げる
PEAK_REL = 0.55           # 線検出の厳しさ（0.45-0.70で調整）
MIN_BAND = 3              # 線として認める連続px幅
BW_THR = 160              # 薄い字なら 130-150 に下げる
INVERT = True             # 白地に黒字想定。逆なら False

ALLOWED_EXT = {".jpg", ".jpeg", ".png", ".pdf"}

def _safe_stem(p: Path) -> str:
    s = p.stem
    s = re.sub(r"[^0-9A-Za-z_\-]+", "_", s)  # 変な文字を _
    s = s.strip("_")
    return s or "file"

def load_first_page_as_image(path: Path) -> Image.Image:
    suf = path.suffix.lower()
    if suf == ".pdf":
        doc = fitz.open(str(path))
        if len(doc) < 1:
            raise RuntimeError("PDFにページがありません。")
        mat = fitz.Matrix(ZOOM_PDF, ZOOM_PDF)
        pix = doc[0].get_pixmap(matrix=mat, alpha=False)
        return Image.frombytes("RGB", [pix.width, pix.height], pix.samples)
    else:
        return Image.open(path).convert("RGB")

def _bands_from_projection(proj: np.ndarray, thresh: float, min_band: int):
    above = proj >= thresh
    bands = []
    i, n = 0, len(proj)
    while i < n:
        if not above[i]:
            i += 1
            continue
        j = i + 1
        while j < n and above[j]:
            j += 1
        if (j - i) >= min_band:
            bands.append((i, j, float(proj[i:j].mean())))
        i = j
    return bands

def _pick_n_best_centers(bands, n):
    if len(bands) < n:
        return []
    bands = sorted(bands, key=lambda x: x[2], reverse=True)[:n]
    centers = [int((a + b) // 2) for a, b, _ in bands]
    centers.sort()
    return centers

def detect_grid_lines(page_img: Image.Image):
    gray = page_img.convert("L")
    arr = np.array(gray)

    # 黒っぽい画素を線として数える（枠線が一番効く）
    mask = arr < LINE_DARK_THRESH
    col_sum = mask.sum(axis=0).astype(np.float32)
    row_sum = mask.sum(axis=1).astype(np.float32)

    col_thresh = PEAK_REL * float(col_sum.max())
    row_thresh = PEAK_REL * float(row_sum.max())

    x_bands = _bands_from_projection(col_sum, col_thresh, MIN_BAND)
    y_bands = _bands_from_projection(row_sum, row_thresh, MIN_BAND)

    x_lines = _pick_n_best_centers(x_bands, COLS + 1)  # 縦線6本
    y_lines = _pick_n_best_centers(y_bands, ROWS + 1)  # 横線5本

    if len(x_lines) != COLS + 1 or len(y_lines) != ROWS + 1:
        raise RuntimeError(f"グリッド線検出失敗（必要: x=6,y=5 / 実際: x={len(x_lines)}, y={len(y_lines)}）")

    return x_lines, y_lines

def split_by_lines(page_img: Image.Image, x_lines, y_lines):
    crops = []
    for r in range(ROWS):
        y0 = y_lines[r] + INNER_MARGIN_PX
        y1 = y_lines[r + 1] - INNER_MARGIN_PX
        for c in range(COLS):
            x0 = x_lines[c] + INNER_MARGIN_PX
            x1 = x_lines[c + 1] - INNER_MARGIN_PX
            crops.append(page_img.crop((x0, y0, x1, y1)))
    return crops

def split_equal_grid(page_img: Image.Image):
    """フォールバック：等間隔分割（授業が止まらない保険）"""
    w, h = page_img.size
    cell_w = w / COLS
    cell_h = h / ROWS
    crops = []
    for r in range(ROWS):
        for c in range(COLS):
            x0 = int(c * cell_w) + INNER_MARGIN_PX
            y0 = int(r * cell_h) + INNER_MARGIN_PX
            x1 = int((c + 1) * cell_w) - INNER_MARGIN_PX
            y1 = int((r + 1) * cell_h) - INNER_MARGIN_PX
            crops.append(page_img.crop((x0, y0, x1, y1)))
    return crops

def to_28x28(img: Image.Image) -> Image.Image:
    g = img.convert("L")
    if INVERT:
        g = ImageOps.invert(g)

    arr = np.array(g)
    bw = (arr > BW_THR).astype(np.uint8) * 255
    bw_img = Image.fromarray(bw, mode="L")

    # 余白除去（白以外があればそこに詰める）
    arr2 = np.array(bw_img)
    ys, xs = np.where(arr2 < 255)
    if len(xs) > 0 and len(ys) > 0:
        x0, x1 = xs.min(), xs.max()
        y0, y1 = ys.min(), ys.max()
        bw_img = bw_img.crop((x0, y0, x1 + 1, y1 + 1))

    bw_img = ImageOps.pad(
        bw_img, (28, 28),
        method=Image.Resampling.NEAREST,
        color=255, centering=(0.5, 0.5)
    )
    return bw_img

def convert_one_file(in_path: Path):
    """
    1ファイルを変換して返す
    戻り:
      info: dict（成功/失敗/注意）
      X: (20,784) float32
      y: (20,) int64
    """
    page_img = load_first_page_as_image(in_path)

    info = {
        "input": str(in_path),
        "mode": None,                 # "line-detect" or "fallback"
        "success": False,
        "message": "",
        "warning": None,
        "error": None,
    }

    try:
        x_lines, y_lines = detect_grid_lines(page_img)
        crops = split_by_lines(page_img, x_lines, y_lines)
        info["mode"] = "line-detect"
        info["success"] = True
        info["message"] = "成功（線検出で分割）"
    except Exception as e:
        # 線検出失敗 → フォールバック
        crops = split_equal_grid(page_img)
        info["mode"] = "fallback"
        info["success"] = True  # 処理自体は継続できる
        info["warning"] = f"{type(e).__name__}: {e}"
        info["message"] = "注意（線検出に失敗 → 等間隔分割で代替）"

    if len(crops) != ROWS * COLS:
        info["success"] = False
        info["error"] = f"分割数が不正です: {len(crops)}（期待: {ROWS*COLS}）"
        info["message"] = "失敗（分割に失敗）"
        return info, None, None

    imgs28 = [to_28x28(c) for c in crops]
    X = np.stack([np.array(im, dtype=np.float32).reshape(-1) / 255.0 for im in imgs28], axis=0)
    y = np.array(LABELS_20, dtype=np.int64)
    return info, X, y

def batch_convert(in_dir: Path, out_dir: Path):
    in_dir = Path(in_dir)
    out_dir = Path(out_dir)
    out_dir.mkdir(parents=True, exist_ok=True)

    targets = sorted([p for p in in_dir.glob("*") if p.suffix.lower() in ALLOWED_EXT])
    if not targets:
        print("対象ファイルがありません:", in_dir)
        return

    ok_count = 0
    warn_count = 0
    fail_count = 0

    for i, p in enumerate(targets, 1):
        info, X, y = convert_one_file(p)

        # 日本語の成功/失敗表示
        head = f"[{i:03d}/{len(targets):03d}] {p.name}"
        if info["success"]:
            ok_count += 1
            if info["mode"] == "fallback":
                warn_count += 1
                print(head, ":", info["message"])
                print("  → 撮り直してください（紙全体・真上・影なし・ピンボケなし）")
                print("  補足:", info["warning"])
            else:
                print(head, ":", info["message"])
        else:
            fail_count += 1
            print(head, ":", info["message"])
            print("  → 撮り直してください（紙全体が写っていない/傾きが大きい可能性）")
            print("  エラー:", info["error"])
            continue

        # 保存（入力ファイルごとに1つのNPZ）
        # ファイル名問題を避ける：連番+安全なstem
        safe = _safe_stem(p)
        out_path = out_dir / f"{i:03d}_{safe}.npz"
        np.savez_compressed(
            out_path,
            X=X.astype(np.float32),
            y=y.astype(np.int64),
            input_name=p.name,
            mode=info["mode"],
            warning="" if info["warning"] is None else info["warning"],
        )

    print("\n===== 変換まとめ =====")
    print("対象:", len(targets))
    print("保存成功:", ok_count)
    print("うち注意（撮り直し推奨/fallback）:", warn_count)
    print("失敗:", fail_count)
    print("保存先:", out_dir)
batch_convert(IN_DIR, OUT_DIR)


モデル検証用コード（手書き数字の写真が認識されるか）

In [ ]:
from pathlib import Path

# ==== 検証データ ====
VERIFY_DIR = Path("data/verify")

# ==== 学習済みモデル ====
MODEL_PATH = Path("data/models/digits_logreg.joblib")

# ==== 日本語フォント ====
FONT_PATH = Path("fonts/NotoSansCJKjp-Regular.otf")

# ==== 可視化設定 ====
SHOW_MAX = 40          # 表示する最大枚数
SHOW_ONLY_MISS = False  # True: 間違いのみ表示 / False: 全体から表示
FILTER_DIGIT = None    # 例: 3 にすると「3だけ表示」、None で全数字

import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

if not FONT_PATH.exists():
    raise FileNotFoundError(f"日本語フォントが見つかりません: {FONT_PATH}")

fm.fontManager.addfont(str(FONT_PATH))
font_prop = fm.FontProperties(fname=str(FONT_PATH))

plt.rcParams["font.family"] = font_prop.get_name()
plt.rcParams["axes.unicode_minus"] = False

print("日本語フォント設定完了:", font_prop.get_name())

import numpy as np

def load_verify_npz(npz_dir: Path):
    files = sorted(npz_dir.glob("*.npz"))
    if not files:
        raise FileNotFoundError(f"verify用 npz が見つかりません: {npz_dir}")

    X_list, y_list, names = [], [], []

    for p in files:
        z = np.load(p, allow_pickle=True)
        X_list.append(z["X"].astype(np.float32))
        y_list.append(z["y"].astype(np.int64))
        names.extend([p.name] * len(z["y"]))

    X = np.concatenate(X_list, axis=0)
    y = np.concatenate(y_list, axis=0)

    return X, y, names, files

Xv, yv, names, npz_files = load_verify_npz(VERIFY_DIR)

print("=== verify 読み込み完了 ===")
print("npz数:", len(npz_files))
print("サンプル数:", len(Xv))
print("ラベル分布:", {i: int((yv == i).sum()) for i in range(10)})

import joblib
from sklearn.metrics import accuracy_score, confusion_matrix

model = joblib.load(MODEL_PATH)

y_pred = model.predict(Xv)

acc = accuracy_score(yv, y_pred)
print("\n=== 検証結果（全体）===")
print(f"全体正答率: {acc:.4f}  (n={len(yv)})")

print("\n=== 検証結果（数字ごと）===")
per_digit_acc = {}
per_digit_n = {}

for d in range(10):
    mask = (yv == d)
    n = int(mask.sum())
    per_digit_n[d] = n
    if n == 0:
        per_digit_acc[d] = None
        print(f"{d}: データなし")
    else:
        a = float((y_pred[mask] == yv[mask]).mean())
        per_digit_acc[d] = a
        print(f"{d}: 正答率={a:.4f}  (n={n})")

import matplotlib.pyplot as plt
import numpy as np

digits = list(range(10))
vals = [0 if per_digit_acc[d] is None else per_digit_acc[d] for d in digits]

plt.figure(figsize=(10, 4))
plt.bar(digits, vals)
plt.ylim(0, 1.0)
plt.xticks(digits, [str(d) for d in digits])
plt.xlabel("数字")
plt.ylabel("正答率")
plt.title("検証データにおける数字ごとの正答率（ロジスティック回帰）")
plt.grid(axis="y", linestyle="--", alpha=0.4)

for d in digits:
    n = per_digit_n[d]
    v = per_digit_acc[d]
    if v is not None:
        plt.text(d, v + 0.02, f"n={n}", ha="center", fontsize=9)

plt.tight_layout()
plt.show()

cm = confusion_matrix(yv, y_pred, labels=digits)

plt.figure(figsize=(6, 5))
plt.imshow(cm, interpolation="nearest")
plt.title("混同行列（検証データ）")
plt.xlabel("予測")
plt.ylabel("正解")
plt.xticks(digits, digits)
plt.yticks(digits, digits)

for i in range(10):
    for j in range(10):
        v = cm[i, j]
        if v != 0:
            plt.text(j, i, str(v), ha="center", va="center", fontsize=9)

plt.colorbar()
plt.tight_layout()
plt.show()

import matplotlib.pyplot as plt

# 表示対象を選別
indices = list(range(len(Xv)))

if SHOW_ONLY_MISS:
    indices = [i for i in indices if y_pred[i] != yv[i]]

if FILTER_DIGIT is not None:
    indices = [i for i in indices if yv[i] == FILTER_DIGIT]

indices = indices[:SHOW_MAX]

print(f"表示数: {len(indices)} / 全体: {len(Xv)}")

# 可視化
cols = 8
rows = (len(indices) + cols - 1) // cols
plt.figure(figsize=(cols * 1.5, rows * 1.8))

for k, i in enumerate(indices):
    ax = plt.subplot(rows, cols, k + 1)
    img = Xv[i].reshape(28, 28)
    ax.imshow(img, cmap="gray")

    pred = y_pred[i]
    true = yv[i]
    ok = (pred == true)

    title = f"予測:{pred}\n正解:{true}"
    if ok:
        ax.set_title(title, color="green", fontsize=9)
    else:
        ax.set_title(title, color="red", fontsize=9)

    ax.axis("off")

plt.suptitle("検証データ：28×28画像と認識結果", fontsize=14)
plt.tight_layout()
plt.show()
